In [ ]:
# Sel ini bisa digunakan untuk instalasi awal jika requirements.txt belum ada atau perlu tambahan cepat
# Namun, idealnya semua dependensi ada di requirements.txt
!pip install flask mysql-connector-python ultralytics pyngrok python-dotenv passlib google-generativeai requests opencv-python numpy google-colab

In [ ]:
# --- BAGIAN 0: FUNGSI SETUP DAN IMPORT (CLOUD) ---
import os
import sys
# import sqlite3 # Not needed if using MySQL from cloud_app
from datetime import datetime
# import torch # YOLO import will handle this
# import cv2 # Imported by cloud_app or app_run as needed
# from ultralytics import YOLO # Model loading handled by app_run logic
from flask import Flask # For type hinting if needed, but app comes from cloud_app
# from passlib.hash import sha256_crypt # Handled by app_run logic
from google.colab import drive
from pyngrok import ngrok, conf
import getpass

print("Pustaka dasar untuk cloud environment berhasil diimpor.")

In [ ]:
# --- BAGIAN 1: KONEKSI GOOGLE DRIVE & PERSIAPAN LINGKUNGAN (CLOUD) ---

print("Menghubungkan Google Drive...")
drive.mount('/content/drive')
print("Google Drive terhubung.")

project_path = '/content/drive/MyDrive/ProjekYOLOv8_Cloud' # GANTI JIKA NAMA FOLDER ANDA BERBEDA
if not os.path.exists(project_path):
    print(f"Error: Project path {project_path} tidak ditemukan. Harap periksa nama folder Anda di Google Drive.")
else:
    os.chdir(project_path)
    print(f"Direktori kerja diubah ke: {os.getcwd()}")
    # Tambahkan project path ke sys.path agar modul Python bisa diimpor
    if project_path not in sys.path:
        sys.path.append(project_path)
        print(f"{project_path} telah ditambahkan ke sys.path")

    print("\nMenginstal dependensi dari requirements.txt...")
    # Pastikan requirements.txt ada di project_path dan berisi semua yang dibutuhkan
    if os.path.exists('requirements.txt'):
        !pip install -q -r requirements.txt
        print("Instalasi dependensi dari requirements.txt selesai.")
    else:
        print("Peringatan: requirements.txt tidak ditemukan. Pastikan semua dependensi terinstal (misalnya dari sel pertama) atau ada di Colab environment.")

    # Membuat direktori yang mungkin dibutuhkan oleh cloud_app atau app_run (jika belum ada)
    os.makedirs('app/static/uploads', exist_ok=True)
    # os.makedirs('app/static/detections', exist_ok=True) # Jika masih digunakan
    print("Direktori untuk 'app/static/uploads' dipastikan ada.")

In [ ]:
# --- BAGIAN 2: KONFIGURASI ENVIRONMENT (CLOUD) ---
print("Konfigurasi Variabel Lingkungan untuk Cloud...")

# Kredensial Database MySQL (untuk cloud_app.py)
os.environ['DB_HOST_CLOUD'] = getpass.getpass('Masukkan Host Database MySQL Cloud: ')
os.environ['DB_USER_CLOUD'] = getpass.getpass('Masukkan User Database MySQL Cloud: ')
os.environ['DB_PASSWORD_CLOUD'] = getpass.getpass('Masukkan Password Database MySQL Cloud: ')
os.environ['DB_NAME_CLOUD'] = input('Masukkan Nama Database MySQL Cloud (default: db_projek_yolo8): ') or 'db_projek_yolo8'

# Kunci API Google Gemini (untuk app_run.py -> get_gemini_description)
os.environ['GOOGLE_GEMINI_API_KEY'] = getpass.getpass('Masukkan Google Gemini API Key (opsional, tekan Enter jika tidak ada): ')

# Secret Key Flask (untuk cloud_app.py)
os.environ['SECRET_KEY'] = getpass.getpass('Masukkan Flask Secret Key (atau biarkan kosong untuk auto-generate): ') or os.urandom(24).hex()

# Path Model YOLO (app_run.py akan menggunakan ini via os.getenv)
os.environ['YOLO_MODEL_OIL_PATH'] = 'models_yolo/best.pt' # Sesuaikan jika path model Anda berbeda
print("Variabel lingkungan telah di-set (atau diminta).")

# Impor app_run untuk memicu pemuatan model global dan konfigurasi awalnya.
try:
    print("\nMengimpor app_run untuk inisialisasi awal (model, GEMINI_API_KEY, dll.)...")
    import app_run 
    print("app_run berhasil diimpor.")
    if hasattr(app_run, 'model_yolo_oil') and app_run.model_yolo_oil:
        print(f"  Model YOLO Oil dari app_run terdeteksi: {type(app_run.model_yolo_oil)}")
    else:
        print("  Peringatan: Model YOLO Oil dari app_run tidak terdeteksi atau None setelah import.")
    if hasattr(app_run, 'GEMINI_API_KEY') and app_run.GEMINI_API_KEY:
        print("  Kunci API Gemini dari app_run terdeteksi.")
    else:
        print("  Peringatan: Kunci API Gemini dari app_run tidak terdeteksi atau None.")
except ImportError as e:
    print(f"Error mengimpor app_run: {e}. Pastikan file ada di {project_path} dan sys.path benar.")
except Exception as e:
    print(f"Error lain saat inisialisasi dari app_run: {e}")

In [ ]:
# --- BAGIAN 3: PERSIAPAN SELESAI, MENJALANKAN APLIKASI DARI cloud_app.py ---
print("\nKonfigurasi lingkungan dan inisialisasi model telah selesai.")
print(f"Direktori kerja saat ini: {os.getcwd()}")
print("Anda sekarang dapat menjalankan aplikasi cloud.")

print("Opsi 1: Jalankan dari terminal Colab (lebih stabil):")
print("  1. Buka terminal baru di Google Colab (File > Open new terminal ATAU ikon '+' lalu 'Terminal').")
print(f"  2. Masuk ke direktori proyek: cd \"{project_path}\"")
print("  3. Menjalankan cloud_app.py: python cloud_app.py")
print("     Anda akan diminta memasukkan token Ngrok jika belum di-set sebagai environment variable NGROK_AUTHTOKEN.")

print("\nOpsi 2: Coba jalankan langsung dari notebook (eksperimental):")
try:
    print("  Mencoba menjalankan cloud_app.py menggunakan %run...\n")
    # Ini akan mengeksekusi blok if __name__ == '__main__': di cloud_app.py
    # Pastikan cloud_app.py dapat diakses dari current working directory.
    %run cloud_app.py 
except Exception as e:
    print(f"  Gagal menjalankan cloud_app.py dari notebook dengan %run: {e}")
    print("  Silakan coba jalankan manual dari terminal Colab seperti instruksi di Opsi 1.")